In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
# import seaborn as sns
%matplotlib inline


SEED=1337

## Сначала создадим и сохраним таблицу с признаками - триграммами слов в датасете

In [28]:
from collections import defaultdict

## Эта таблица лежит в папке с ДЗ, поэтому можно не запускать код до следующего заголовка, чтобы не генерировать ее заново

In [117]:
# df = pd.read_csv('ner_short.csv', index_col=0)

In [116]:
# df

In [103]:
# sentence length
# tdf = df.set_index('sentence_idx')
# tdf['length'] = df.groupby('sentence_idx').tag.count()
# df = tdf.reset_index(drop=False)

In [104]:
# encode categorial variables

# le = LabelEncoder()
# df['pos'] = le.fit_transform(df.pos)
# df['next-pos'] = le.fit_transform(df['next-pos'])
# df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
# df['prev-pos'] = le.fit_transform(df['prev-pos'])
# df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [105]:
# df['isStart'] = [int('START' in i) for i in df['prev-word']]

In [106]:
# df['isUpper'] = [int(i[0].isupper()) for i in df.word]

In [107]:
# bigrams = []
# n=3
# for b in df.word:
#     bigrams+=[b[i:i+n] for i in range(len(b)-n+1)]
# fin_bigr = set(bigrams)

In [108]:
# matrix_df = defaultdict(list)
# for word in df.word:
#     for bigr in fin_bigr:
#         if bigr in word:
#             matrix_df[bigr].append(1)
#         else:
#             matrix_df[bigr].append(0)
# dict_df = dict(matrix_df)
# bigr_features = df.from_dict(dict_df)

In [109]:
# joined_df = df.join(bigr_features, rsuffix='_3gr')

In [110]:
# joined_df.head()

In [111]:
# joined_df.shape

In [112]:
# joined_df.to_csv('joined_df.csv')

# Отсюда начинается обучение

## Признаки: w2v embedding + 3-grams of words

In [29]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
# import seaborn as sns
%matplotlib inline


SEED=1337

In [22]:
df = pd.read_csv('/Users/Valeriya/Downloads/joined_df.csv', index_col=0) ## тут надо открыть таблицу,
## которая была создана в предыдущем разделе

In [30]:
# df_fst = pd.read_csv('ner_short.csv')

In [31]:
# df = pd.read_csv('/Users/Valeriya/Downloads/3grams.csv', index_col=0)

In [32]:
# df.head()

In [33]:
# splitting
y = LabelEncoder().fit_transform(df_fst.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape)
print('test', df_test.shape)

train (50155, 4744)
test (16719, 4744)


In [34]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict
from glove import Corpus, Glove

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    

class GloveWrapper(TransformerMixin):
    def __init__(self, window=5, learning_rate=0.05, size=100, epochs=100, random_state=SEED, verbose=False):
        self.window_ = window
        self.learning_rate_ = learning_rate
        self.size_ = size
        self.epochs_ = epochs
        self.glove = None
        self.random_state = random_state
        self.verbose_ = verbose
    
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        corpus = Corpus()
        corpus.fit(sentences_list, window=self.window_)
        self.glove = Glove(no_components=self.size_, 
                           learning_rate=self.learning_rate_, 
                           random_state=self.random_state)
        self.glove.fit(corpus.matrix, 
                  epochs=self.epochs_, 
                  no_threads=4, verbose=self.verbose_)
        self.glove.add_dictionary(corpus.dictionary)
        return self
    
    def has(self, word):
        return word in self.glove.dictionary

    def transform(self, X):
        """
        X: list of words
        """
        if self.glove is None:
            raise Exception('model not fitted')
            
        return np.array([self.glove.word_vectors[self.glove.dictionary[w]] if w in self.glove.dictionary else np.zeros(self.size_) 
                         for w in X])

In [35]:
%%time
# we have to fit embedding models on whole dataset as they depend on word order
from gensim.models.word2vec import Word2Vec


# notice, that our dataset has window=2
sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 40.2 s, sys: 360 ms, total: 40.5 s
Wall time: 15 s


In [36]:
%%time


w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=False, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 1min 12s, sys: 311 ms, total: 1min 13s
Wall time: 25.1 s


In [37]:
%%time


w2v_gl = GloveWrapper(window=2, learning_rate=0.05, size=300, epochs=300, random_state=SEED, verbose=False)
w2v_gl.fit(sentences_list)

CPU times: user 7min 1s, sys: 906 ms, total: 7min 2s
Wall time: 1min 48s


In [8]:
# %%time
# # baseline 1 
# # random labels
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.dummy import DummyClassifier


# columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

# model = Pipeline([
#     ('enc', OneHotEncoder()),
#     ('est', DummyClassifier(random_state=SEED)),
# ])

# model.fit(df_train[columns], y_train)

# print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
# print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

In [9]:
# %%time
# # baseline 2 
# # pos features + one hot encoding + logistic regression
# from sklearn.preprocessing import OneHotEncoder


# columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

# model = Pipeline([
#     ('enc', OneHotEncoder()),
#     ('est', LogisticRegressionCV(Cs=5, cv=5, n_jobs=-1, scoring='f1_macro', 
#                              penalty='l2', solver='newton-cg', multi_class='multinomial', random_state=SEED, verbose=True)),
# ])

# model.fit(df_train[columns], y_train)

# print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
# print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

In [10]:
# %%time
# # baseline 3
# # use word2vec cbow embedding + baseline 2 + svm
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# from sklearn.svm import LinearSVC
# import scipy.sparse as sp
# embeding = w2v_gl
# encoder_pos = OneHotEncoder()
# X_train = sp.hstack([
#     embeding.transform(df_train.word),
#     embeding.transform(df_train['next-word']),
#     embeding.transform(df_train['next-next-word']),
#     embeding.transform(df_train['prev-word']),
#     embeding.transform(df_train['prev-prev-word']),
#     encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
# ])
# X_test = sp.hstack([
#     embeding.transform(df_test.word),
#     embeding.transform(df_test['next-word']),
#     embeding.transform(df_test['next-next-word']),
#     embeding.transform(df_test['prev-word']),
#     embeding.transform(df_test['prev-prev-word']),
#     encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
# ])

# model = LinearSVC()
# model.fit(X_train, y_train)

# print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
# print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

In [40]:
from sklearn.grid_search import GridSearchCV

## А теперь добавляем свои фичи

In [38]:
%%time
# baseline 3
# use word2vec cbow embedding + baseline 2 + svm
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
import scipy.sparse as sp
embeding = w2v_gl
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    df_train.loc[:, 'isStart':].as_matrix(),
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    df_test.loc[:, 'isStart':].as_matrix(),
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

CPU times: user 12.9 s, sys: 9 s, total: 21.9 s
Wall time: 25.6 s


In [93]:
%%time
model = LinearSVC(random_state=SEED)

CPU times: user 16 µs, sys: 2 µs, total: 18 µs
Wall time: 21.9 µs


In [94]:
Cs = np.logspace(0, -1.3, 3)

In [118]:
## Подбираю параметр С

In [95]:
model_grid = GridSearchCV(model, param_grid={'C': Cs})

In [96]:
model_grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1337, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([ 1.     ,  0.22387,  0.05012])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [97]:
best_est = model_grid.best_estimator_ # лучшая модель

In [99]:
print('train', metrics.f1_score(y_train, best_est.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, best_est.predict(X_test), average='macro'))

train 0.977355363233
test 0.83910346672


## test 0.83910346672